<a href="https://colab.research.google.com/github/habibanalytics/University_Final_Year_Project/blob/master/Notebooks/Chicago_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Objective of this preprocessing is to create encoded dataset for predicting Secondary Crime Description**


**Avoid colab clossing**
```
function ClickConnect(){
    document.querySelector("colab-connect-button").click()
    console.log("Clicked on connect button"); 
}
setInterval(ClickConnect,60000)
```

# **Imports**

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# mounting google drive

In [20]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# **Importing Dataset**

In [21]:
file_path= "/content/gdrive/My Drive/Final Year Project/Dataset/dataset_analyzed.csv"
data= pd.read_csv(file_path) #, nrows= 10000
df= data.copy()

In [22]:
df.info()
print("Rows and Columns size: ", df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198174 entries, 0 to 198173
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   date_of_occurrence     198174 non-null  object 
 1   block                  198174 non-null  object 
 2   iucr                   198174 non-null  object 
 3   primary_description    198174 non-null  object 
 4   secondary_description  198174 non-null  object 
 5   location_description   198174 non-null  object 
 6   arrest                 198174 non-null  object 
 7   domestic               198174 non-null  object 
 8   beat                   198174 non-null  int64  
 9   ward                   198174 non-null  float64
 10  fbi_cd                 198174 non-null  object 
 11  latitude               198174 non-null  float64
 12  longitude              198174 non-null  float64
 13  month                  198174 non-null  object 
 14  week_day               198174 non-nu

# Changing Data type

In [23]:
df["ward"]= df["ward"].astype(object)
df["beat"]= df["beat"].astype(object)
df["hour"]= df["hour"].astype(object)
df["year"]= df["year"].astype(object)

In [24]:
df.head(2)

,date_of_occurrence,block,iucr,primary_description,secondary_description,location_description,arrest,domestic,beat,ward,fbi_cd,latitude,longitude,month,week_day,hour,year
0,2020-02-19 05:15:00,079XX S WABASH AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,N,Y,623,6,08B,41.750225,-87.622976,Feb,Wednesday,5,2020
1,2020-02-19 05:25:00,0000X E LOWER WACKER PL,0486,BATTERY,DOMESTIC BATTERY SIMPLE,HOTEL/MOTEL,N,Y,111,42,08B,41.886815,-87.625626,Feb,Wednesday,5,2020


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198174 entries, 0 to 198173
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   date_of_occurrence     198174 non-null  object 
 1   block                  198174 non-null  object 
 2   iucr                   198174 non-null  object 
 3   primary_description    198174 non-null  object 
 4   secondary_description  198174 non-null  object 
 5   location_description   198174 non-null  object 
 6   arrest                 198174 non-null  object 
 7   domestic               198174 non-null  object 
 8   beat                   198174 non-null  object 
 9   ward                   198174 non-null  object 
 10  fbi_cd                 198174 non-null  object 
 11  latitude               198174 non-null  float64
 12  longitude              198174 non-null  float64
 13  month                  198174 non-null  object 
 14  week_day               198174 non-nu

In [26]:
del df["date_of_occurrence"]
del df["arrest"]
del df["iucr"]
del df["latitude"]
del df["longitude"]

# **One hot encoding for features with many variables**
In this i will pick up top most freqquencty occuring labels from each feature and for the rest of the labels if they occur in the data would have their complete row as zero and they will not contain a column of their own


## Counting columns and unique values for each column before encoding

[onehotencoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)

In [27]:
orig_rows, orig_col= df.shape
print(" Rows",orig_rows,"\n","Columns",orig_col)

 Rows 198174 
 Columns 12


In [28]:
for column in df.columns:
    print(column, ": ", df[column].nunique())
print("Total rows: ", orig_rows)

block :  26706
primary_description :  33
secondary_description :  374
location_description :  163
domestic :  2
beat :  274
ward :  50
fbi_cd :  26
month :  12
week_day :  7
hour :  24
year :  2
Total rows:  198174


In [29]:
df.columns

Index(['block', 'primary_description', 'secondary_description',
       'location_description', 'domestic', 'beat', 'ward', 'fbi_cd', 'month',
       'week_day', 'hour', 'year'],
      dtype='object')

# **Data For Predicting Secondary Type**

# **Which values to pick?**

, block, beat, location_description, ,domestic, beat, ward, fbi_cd, month, week_day, hour


**Top Values :**

* top 12 **primary_description**
description
* top 20 wards
    * top 30 **blocks** from top wards
    * top 2 **beats** from top wards
* top 30 **location description**
* top 10 **FBI code**

**Others**

* domestic :  2
* month :  12
* week_day :  7
* hour :  24



# **Encoder Function**

This function accepts:
* **column_name**: The column to encode
* **top_values**: List of labels to encode

In [30]:
def encoder_function(column_name, labels):
    """
    This function accepts:
    column_name: The column to encode
    labels: List of values to encode
    """
    for category in labels:
        new_col= column_name + "_"+ category
        df[new_col]= np.where(df[column_name] == category, 1, 0)

# **Getting internal top 10 values function**

In [31]:
def list_inner_top_values(column_name, outer_top, no_of_top_topick, Column_to_count):
    """
    This function accepts:
    column_name: The name of the outer level column
    outer_top: (List) top values of outer column
    no_of_top_topick: Number of top values to pick from inner column
    Column_to_count: Name of column from which you want to pick top values of
    """
    top_values=[]
    for value in outer_top:
        filt= (df[column_name] == value )
        top_values.extend([x for x in df[filt][Column_to_count].value_counts(ascending= False).head(no_of_top_topick).index])
    return top_values

# **Top Labels and their encoding**

## **Top 10 primary description**

In [32]:
n_values= 12
top_prim_des= [x for x in df["primary_description"].value_counts(ascending= False).head(n_values).index]
top_values = top_prim_des.copy()

### **Encoding Top Primary Description values**

In [33]:
column_name= "primary_description"
encoder_function(column_name, top_values)

In [34]:
df.head(2)

,block,primary_description,secondary_description,location_description,domestic,beat,ward,fbi_cd,month,week_day,hour,year,primary_description_BATTERY,primary_description_THEFT,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_MOTOR VEHICLE THEFT,primary_description_WEAPONS VIOLATION,primary_description_BURGLARY,primary_description_ROBBERY,primary_description_NARCOTICS,primary_description_CRIMINAL TRESPASS
0,079XX S WABASH AVE,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,Y,623,6,08B,Feb,Wednesday,5,2020,1,0,0,0,0,0,0,0,0,0,0,0
1,0000X E LOWER WACKER PL,BATTERY,DOMESTIC BATTERY SIMPLE,HOTEL/MOTEL,Y,111,42,08B,Feb,Wednesday,5,2020,1,0,0,0,0,0,0,0,0,0,0,0


## **Top Wards**

In [35]:
n_values= 20
column_= "ward"
top_wards= [x for x in df[column_].value_counts(ascending= False).head(n_values).index]
top_values = list(map(str,top_wards))

### **Encoding Wards**

In [36]:
column_name= "ward"
encoder_function(column_name, top_values)

In [37]:
df.head(2)

,block,primary_description,secondary_description,location_description,domestic,beat,ward,fbi_cd,month,week_day,hour,year,primary_description_BATTERY,primary_description_THEFT,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_MOTOR VEHICLE THEFT,primary_description_WEAPONS VIOLATION,primary_description_BURGLARY,primary_description_ROBBERY,primary_description_NARCOTICS,primary_description_CRIMINAL TRESPASS,ward_28.0,ward_27.0,ward_24.0,ward_6.0,ward_8.0,ward_17.0,ward_42.0,ward_9.0,ward_21.0,ward_20.0,ward_16.0,ward_7.0,ward_34.0,ward_37.0,ward_3.0,ward_29.0,ward_4.0,ward_5.0,ward_25.0,ward_10.0
0,079XX S WABASH AVE,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,Y,623,6,08B,Feb,Wednesday,5,2020,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0000X E LOWER WACKER PL,BATTERY,DOMESTIC BATTERY SIMPLE,HOTEL/MOTEL,Y,111,42,08B,Feb,Wednesday,5,2020,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## **Top Blocks from top wards**

In [38]:
# Getting list of blocks that occur in top wards
# to pass
column_name= "ward" # The name of the outer level column
outer_top= top_wards # Top values of outer column
no_of_top_topick =10 #these are how many number of values to pick for the inner top values
Column_to_count= "block" # Name of column from which you want to pick top values of

top_values= list(set(list_inner_top_values(column_name, outer_top, no_of_top_topick, Column_to_count)))
print(top_values)

['033XX W MAYPOLE AVE', '004XX N MICHIGAN AVE', '006XX E 131ST ST', '021XX E 71ST ST', '012XX S ASHLAND AVE', '039XX W GLADYS AVE', '0000X S STATE ST', '002XX W 108TH ST', '051XX W NORTH AVE', '083XX S HOLLAND RD', '002XX W 63RD ST', '079XX S HALSTED ST', '0000X W 71ST ST', '075XX S SOUTH SHORE DR', '033XX W FILLMORE ST', '011XX S STATE ST', '039XX W VAN BUREN ST', '043XX S MICHIGAN AVE', '029XX S FEDERAL ST', '063XX S ARTESIAN AVE', '010XX N PULASKI RD', '016XX S HOMAN AVE', '089XX S HOUSTON AVE', '062XX S WESTERN AVE', '012XX S WABASH AVE', '005XX N STATE ST', '082XX S MARYLAND AVE', '057XX W MADISON ST', '002XX E HURON ST', '020XX S MICHIGAN AVE', '061XX S KIMBARK AVE', '013XX S THROOP ST', '094XX S ASHLAND AVE', '098XX S STATE ST', '056XX W MADISON ST', '079XX S ASHLAND AVE', '109XX S DOTY AVE W', '034XX W FRANKLIN BLVD', '074XX S COLFAX AVE', '0000X W 95TH ST', '067XX S EAST END AVE', '076XX S STEWART AVE', '005XX E 79TH ST', '006XX W HARRISON ST', '098XX S HALSTED ST', '001XX W L

### **Encoding Blocks**

In [39]:
column_name= "block"
encoder_function(column_name, top_values)

In [40]:
df.head(2)

,block,primary_description,secondary_description,location_description,domestic,beat,ward,fbi_cd,month,week_day,hour,year,primary_description_BATTERY,primary_description_THEFT,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_MOTOR VEHICLE THEFT,primary_description_WEAPONS VIOLATION,primary_description_BURGLARY,primary_description_ROBBERY,primary_description_NARCOTICS,primary_description_CRIMINAL TRESPASS,ward_28.0,ward_27.0,ward_24.0,ward_6.0,ward_8.0,ward_17.0,ward_42.0,ward_9.0,ward_21.0,ward_20.0,ward_16.0,ward_7.0,ward_34.0,ward_37.0,ward_3.0,ward_29.0,...,block_078XX S HALSTED ST,block_010XX E 132ND ST,block_081XX S MAY ST,block_079XX S DREXEL AVE,block_063XX S DR MARTIN LUTHER KING JR DR,block_029XX S DEARBORN ST,block_034XX S DR MARTIN LUTHER KING JR DR,block_050XX S MICHIGAN AVE,block_013XX W HASTINGS ST,block_013XX W 76TH ST,block_075XX S STONY ISLAND AVE,block_066XX S HALSTED ST,block_002XX W GARFIELD BLVD,block_071XX S JEFFERY BLVD,block_052XX S DREXEL AVE,block_035XX W CHICAGO AVE,block_082XX S ELLIS AVE,block_064XX S LOWE AVE,block_037XX W CHICAGO AVE,block_0000X E WACKER DR,block_077XX S SOUTH SHORE DR,block_008XX W 115TH ST,block_011XX S CANAL ST,block_099XX S HALSTED ST,block_050XX S EAST END AVE,block_076XX S HALSTED ST,block_070XX S ASHLAND AVE,block_029XX W HARRISON ST,block_006XX N MICHIGAN AVE,block_117XX S MARSHFIELD AVE,block_039XX W THOMAS ST,block_015XX S DRAKE AVE,block_063XX S RACINE AVE,block_056XX W WASHINGTON BLVD,block_006XX S CENTRAL AVE,block_034XX W ROOSEVELT RD,block_007XX E 111TH ST,block_064XX S DR MARTIN LUTHER KING JR DR,block_0000X S SPRINGFIELD AVE,block_0000X W 79TH ST
0,079XX S WABASH AVE,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,Y,623,6,08B,Feb,Wednesday,5,2020,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0000X E LOWER WACKER PL,BATTERY,DOMESTIC BATTERY SIMPLE,HOTEL/MOTEL,Y,111,42,08B,Feb,Wednesday,5,2020,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## **Top beats from top wards**

In [41]:
# Getting list of blocks that occur in top wards
# to pass
column_name= "ward" # The name of the outer level column
outer_top= top_wards # Top values of outer column
no_of_top_topick =2 #these are how many number of values to pick for the inner top values
Column_to_count= "beat" # Name of column from which you want to pick top values of

top_values= list(set(list_inner_top_values(column_name, outer_top, no_of_top_topick, Column_to_count)))
print(top_values)

[131, 522, 524, 533, 414, 1533, 421, 1831, 423, 424, 1834, 2223, 432, 312, 313, 825, 713, 331, 332, 1233, 1234, 213, 1112, 1113, 222, 1121, 612, 613, 2533, 2532, 1511, 621, 623, 624, 1522, 1011, 631, 123, 1021, 511]


#### **Encoding Top Beats**

In [42]:
column_name= "beat"
top_values= list(map(str,top_values))
encoder_function(column_name, top_values)

In [43]:
df.head(2)

,block,primary_description,secondary_description,location_description,domestic,beat,ward,fbi_cd,month,week_day,hour,year,primary_description_BATTERY,primary_description_THEFT,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_MOTOR VEHICLE THEFT,primary_description_WEAPONS VIOLATION,primary_description_BURGLARY,primary_description_ROBBERY,primary_description_NARCOTICS,primary_description_CRIMINAL TRESPASS,ward_28.0,ward_27.0,ward_24.0,ward_6.0,ward_8.0,ward_17.0,ward_42.0,ward_9.0,ward_21.0,ward_20.0,ward_16.0,ward_7.0,ward_34.0,ward_37.0,ward_3.0,ward_29.0,...,beat_131,beat_522,beat_524,beat_533,beat_414,beat_1533,beat_421,beat_1831,beat_423,beat_424,beat_1834,beat_2223,beat_432,beat_312,beat_313,beat_825,beat_713,beat_331,beat_332,beat_1233,beat_1234,beat_213,beat_1112,beat_1113,beat_222,beat_1121,beat_612,beat_613,beat_2533,beat_2532,beat_1511,beat_621,beat_623,beat_624,beat_1522,beat_1011,beat_631,beat_123,beat_1021,beat_511
0,079XX S WABASH AVE,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,Y,623,6,08B,Feb,Wednesday,5,2020,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0000X E LOWER WACKER PL,BATTERY,DOMESTIC BATTERY SIMPLE,HOTEL/MOTEL,Y,111,42,08B,Feb,Wednesday,5,2020,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## **Top Locations**

In [44]:
top_prim_des= [x for x in df["location_description"].value_counts(ascending= False).head(50).index]
top_values = top_prim_des.copy()
top_values

['STREET',
 'RESIDENCE',
 'APARTMENT',
 'SIDEWALK',
 'PARKING LOT / GARAGE (NON RESIDENTIAL)',
 'OTHER (SPECIFY)',
 'SMALL RETAIL STORE',
 'ALLEY',
 'RESIDENCE - YARD (FRONT / BACK)',
 'RESIDENCE - PORCH / HALLWAY',
 'GAS STATION',
 'RESTAURANT',
 'VEHICLE NON-COMMERCIAL',
 'RESIDENCE - GARAGE',
 'GROCERY FOOD STORE',
 'DEPARTMENT STORE',
 'CONVENIENCE STORE',
 'COMMERCIAL / BUSINESS OFFICE',
 'DRUG STORE',
 'CTA TRAIN',
 'HOSPITAL BUILDING / GROUNDS',
 'HOTEL / MOTEL',
 'PARK PROPERTY',
 'BANK',
 'CHA APARTMENT',
 'CTA BUS',
 'POLICE FACILITY / VEHICLE PARKING LOT',
 'NURSING / RETIREMENT HOME',
 'DRIVEWAY - RESIDENTIAL',
 'VACANT LOT / LAND',
 'CTA STATION',
 'BAR OR TAVERN',
 'CTA PLATFORM',
 'CHA PARKING LOT / GROUNDS',
 'SCHOOL - PUBLIC BUILDING',
 'TAVERN / LIQUOR STORE',
 'CTA BUS STOP',
 'GOVERNMENT BUILDING / PROPERTY',
 'CHURCH / SYNAGOGUE / PLACE OF WORSHIP',
 'OTHER',
 'MEDICAL / DENTAL OFFICE',
 'CURRENCY EXCHANGE',
 'AUTO / BOAT / RV DEALERSHIP',
 'CONSTRUCTION SITE',
 'S

### **Encoding Top Locations**

In [45]:
column_name= "location_description"
encoder_function(column_name, top_values)

In [46]:
df.head(2)

,block,primary_description,secondary_description,location_description,domestic,beat,ward,fbi_cd,month,week_day,hour,year,primary_description_BATTERY,primary_description_THEFT,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_MOTOR VEHICLE THEFT,primary_description_WEAPONS VIOLATION,primary_description_BURGLARY,primary_description_ROBBERY,primary_description_NARCOTICS,primary_description_CRIMINAL TRESPASS,ward_28.0,ward_27.0,ward_24.0,ward_6.0,ward_8.0,ward_17.0,ward_42.0,ward_9.0,ward_21.0,ward_20.0,ward_16.0,ward_7.0,ward_34.0,ward_37.0,ward_3.0,ward_29.0,...,location_description_GAS STATION,location_description_RESTAURANT,location_description_VEHICLE NON-COMMERCIAL,location_description_RESIDENCE - GARAGE,location_description_GROCERY FOOD STORE,location_description_DEPARTMENT STORE,location_description_CONVENIENCE STORE,location_description_COMMERCIAL / BUSINESS OFFICE,location_description_DRUG STORE,location_description_CTA TRAIN,location_description_HOSPITAL BUILDING / GROUNDS,location_description_HOTEL / MOTEL,location_description_PARK PROPERTY,location_description_BANK,location_description_CHA APARTMENT,location_description_CTA BUS,location_description_POLICE FACILITY / VEHICLE PARKING LOT,location_description_NURSING / RETIREMENT HOME,location_description_DRIVEWAY - RESIDENTIAL,location_description_VACANT LOT / LAND,location_description_CTA STATION,location_description_BAR OR TAVERN,location_description_CTA PLATFORM,location_description_CHA PARKING LOT / GROUNDS,location_description_SCHOOL - PUBLIC BUILDING,location_description_TAVERN / LIQUOR STORE,location_description_CTA BUS STOP,location_description_GOVERNMENT BUILDING / PROPERTY,location_description_CHURCH / SYNAGOGUE / PLACE OF WORSHIP,location_description_OTHER,location_description_MEDICAL / DENTAL OFFICE,location_description_CURRENCY EXCHANGE,location_description_AUTO / BOAT / RV DEALERSHIP,location_description_CONSTRUCTION SITE,location_description_SCHOOL - PUBLIC GROUNDS,location_description_ATM (AUTOMATIC TELLER MACHINE),location_description_PARKING LOT/GARAGE(NON.RESID.),location_description_WAREHOUSE,"location_description_SCHOOL, PUBLIC, BUILDING",location_description_BARBERSHOP
0,079XX S WABASH AVE,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,Y,623,6,08B,Feb,Wednesday,5,2020,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0000X E LOWER WACKER PL,BATTERY,DOMESTIC BATTERY SIMPLE,HOTEL/MOTEL,Y,111,42,08B,Feb,Wednesday,5,2020,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## **Top FBI Code**

In [47]:
top_prim_des= [x for x in df["fbi_cd"].value_counts(ascending= False).head(10).index]
top_values = top_prim_des.copy()
top_values

['06', '08B', '14', '11', '08A', '26', '07', '15', '05', '04B']

### **Encoding top FBI codes**

In [48]:
column_name= "fbi_cd"
encoder_function(column_name, top_values)

In [49]:
df.head(2)

,block,primary_description,secondary_description,location_description,domestic,beat,ward,fbi_cd,month,week_day,hour,year,primary_description_BATTERY,primary_description_THEFT,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_MOTOR VEHICLE THEFT,primary_description_WEAPONS VIOLATION,primary_description_BURGLARY,primary_description_ROBBERY,primary_description_NARCOTICS,primary_description_CRIMINAL TRESPASS,ward_28.0,ward_27.0,ward_24.0,ward_6.0,ward_8.0,ward_17.0,ward_42.0,ward_9.0,ward_21.0,ward_20.0,ward_16.0,ward_7.0,ward_34.0,ward_37.0,ward_3.0,ward_29.0,...,location_description_HOSPITAL BUILDING / GROUNDS,location_description_HOTEL / MOTEL,location_description_PARK PROPERTY,location_description_BANK,location_description_CHA APARTMENT,location_description_CTA BUS,location_description_POLICE FACILITY / VEHICLE PARKING LOT,location_description_NURSING / RETIREMENT HOME,location_description_DRIVEWAY - RESIDENTIAL,location_description_VACANT LOT / LAND,location_description_CTA STATION,location_description_BAR OR TAVERN,location_description_CTA PLATFORM,location_description_CHA PARKING LOT / GROUNDS,location_description_SCHOOL - PUBLIC BUILDING,location_description_TAVERN / LIQUOR STORE,location_description_CTA BUS STOP,location_description_GOVERNMENT BUILDING / PROPERTY,location_description_CHURCH / SYNAGOGUE / PLACE OF WORSHIP,location_description_OTHER,location_description_MEDICAL / DENTAL OFFICE,location_description_CURRENCY EXCHANGE,location_description_AUTO / BOAT / RV DEALERSHIP,location_description_CONSTRUCTION SITE,location_description_SCHOOL - PUBLIC GROUNDS,location_description_ATM (AUTOMATIC TELLER MACHINE),location_description_PARKING LOT/GARAGE(NON.RESID.),location_description_WAREHOUSE,"location_description_SCHOOL, PUBLIC, BUILDING",location_description_BARBERSHOP,fbi_cd_06,fbi_cd_08B,fbi_cd_14,fbi_cd_11,fbi_cd_08A,fbi_cd_26,fbi_cd_07,fbi_cd_15,fbi_cd_05,fbi_cd_04B
0,079XX S WABASH AVE,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,Y,623,6,08B,Feb,Wednesday,5,2020,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0000X E LOWER WACKER PL,BATTERY,DOMESTIC BATTERY SIMPLE,HOTEL/MOTEL,Y,111,42,08B,Feb,Wednesday,5,2020,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


# **Label Encoding of other  columns**

## **Encoding domestic Column**

In [50]:
column_name= "domestic"

labels = list(df[column_name].unique())
encoder_function(column_name, labels)
df.head(2)

,block,primary_description,secondary_description,location_description,domestic,beat,ward,fbi_cd,month,week_day,hour,year,primary_description_BATTERY,primary_description_THEFT,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_MOTOR VEHICLE THEFT,primary_description_WEAPONS VIOLATION,primary_description_BURGLARY,primary_description_ROBBERY,primary_description_NARCOTICS,primary_description_CRIMINAL TRESPASS,ward_28.0,ward_27.0,ward_24.0,ward_6.0,ward_8.0,ward_17.0,ward_42.0,ward_9.0,ward_21.0,ward_20.0,ward_16.0,ward_7.0,ward_34.0,ward_37.0,ward_3.0,ward_29.0,...,location_description_PARK PROPERTY,location_description_BANK,location_description_CHA APARTMENT,location_description_CTA BUS,location_description_POLICE FACILITY / VEHICLE PARKING LOT,location_description_NURSING / RETIREMENT HOME,location_description_DRIVEWAY - RESIDENTIAL,location_description_VACANT LOT / LAND,location_description_CTA STATION,location_description_BAR OR TAVERN,location_description_CTA PLATFORM,location_description_CHA PARKING LOT / GROUNDS,location_description_SCHOOL - PUBLIC BUILDING,location_description_TAVERN / LIQUOR STORE,location_description_CTA BUS STOP,location_description_GOVERNMENT BUILDING / PROPERTY,location_description_CHURCH / SYNAGOGUE / PLACE OF WORSHIP,location_description_OTHER,location_description_MEDICAL / DENTAL OFFICE,location_description_CURRENCY EXCHANGE,location_description_AUTO / BOAT / RV DEALERSHIP,location_description_CONSTRUCTION SITE,location_description_SCHOOL - PUBLIC GROUNDS,location_description_ATM (AUTOMATIC TELLER MACHINE),location_description_PARKING LOT/GARAGE(NON.RESID.),location_description_WAREHOUSE,"location_description_SCHOOL, PUBLIC, BUILDING",location_description_BARBERSHOP,fbi_cd_06,fbi_cd_08B,fbi_cd_14,fbi_cd_11,fbi_cd_08A,fbi_cd_26,fbi_cd_07,fbi_cd_15,fbi_cd_05,fbi_cd_04B,domestic_Y,domestic_N
0,079XX S WABASH AVE,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,Y,623,6,08B,Feb,Wednesday,5,2020,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,0000X E LOWER WACKER PL,BATTERY,DOMESTIC BATTERY SIMPLE,HOTEL/MOTEL,Y,111,42,08B,Feb,Wednesday,5,2020,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0


## **Encoding month Column**

In [51]:
column_name= "month"

labels = list(df[column_name].unique())
encoder_function(column_name, labels)
df.head(2)

,block,primary_description,secondary_description,location_description,domestic,beat,ward,fbi_cd,month,week_day,hour,year,primary_description_BATTERY,primary_description_THEFT,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_MOTOR VEHICLE THEFT,primary_description_WEAPONS VIOLATION,primary_description_BURGLARY,primary_description_ROBBERY,primary_description_NARCOTICS,primary_description_CRIMINAL TRESPASS,ward_28.0,ward_27.0,ward_24.0,ward_6.0,ward_8.0,ward_17.0,ward_42.0,ward_9.0,ward_21.0,ward_20.0,ward_16.0,ward_7.0,ward_34.0,ward_37.0,ward_3.0,ward_29.0,...,location_description_SCHOOL - PUBLIC BUILDING,location_description_TAVERN / LIQUOR STORE,location_description_CTA BUS STOP,location_description_GOVERNMENT BUILDING / PROPERTY,location_description_CHURCH / SYNAGOGUE / PLACE OF WORSHIP,location_description_OTHER,location_description_MEDICAL / DENTAL OFFICE,location_description_CURRENCY EXCHANGE,location_description_AUTO / BOAT / RV DEALERSHIP,location_description_CONSTRUCTION SITE,location_description_SCHOOL - PUBLIC GROUNDS,location_description_ATM (AUTOMATIC TELLER MACHINE),location_description_PARKING LOT/GARAGE(NON.RESID.),location_description_WAREHOUSE,"location_description_SCHOOL, PUBLIC, BUILDING",location_description_BARBERSHOP,fbi_cd_06,fbi_cd_08B,fbi_cd_14,fbi_cd_11,fbi_cd_08A,fbi_cd_26,fbi_cd_07,fbi_cd_15,fbi_cd_05,fbi_cd_04B,domestic_Y,domestic_N,month_Feb,month_Mar,month_Apr,month_May,month_Jun,month_Jul,month_Aug,month_Sep,month_Oct,month_Nov,month_Dec,month_Jan
0,079XX S WABASH AVE,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,Y,623,6,08B,Feb,Wednesday,5,2020,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0000X E LOWER WACKER PL,BATTERY,DOMESTIC BATTERY SIMPLE,HOTEL/MOTEL,Y,111,42,08B,Feb,Wednesday,5,2020,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0


## **Encoding weekday Column**

In [52]:
column_name= "week_day"

labels = list(df[column_name].unique())
encoder_function(column_name, labels)
df.head(2)

,block,primary_description,secondary_description,location_description,domestic,beat,ward,fbi_cd,month,week_day,hour,year,primary_description_BATTERY,primary_description_THEFT,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_MOTOR VEHICLE THEFT,primary_description_WEAPONS VIOLATION,primary_description_BURGLARY,primary_description_ROBBERY,primary_description_NARCOTICS,primary_description_CRIMINAL TRESPASS,ward_28.0,ward_27.0,ward_24.0,ward_6.0,ward_8.0,ward_17.0,ward_42.0,ward_9.0,ward_21.0,ward_20.0,ward_16.0,ward_7.0,ward_34.0,ward_37.0,ward_3.0,ward_29.0,...,location_description_CURRENCY EXCHANGE,location_description_AUTO / BOAT / RV DEALERSHIP,location_description_CONSTRUCTION SITE,location_description_SCHOOL - PUBLIC GROUNDS,location_description_ATM (AUTOMATIC TELLER MACHINE),location_description_PARKING LOT/GARAGE(NON.RESID.),location_description_WAREHOUSE,"location_description_SCHOOL, PUBLIC, BUILDING",location_description_BARBERSHOP,fbi_cd_06,fbi_cd_08B,fbi_cd_14,fbi_cd_11,fbi_cd_08A,fbi_cd_26,fbi_cd_07,fbi_cd_15,fbi_cd_05,fbi_cd_04B,domestic_Y,domestic_N,month_Feb,month_Mar,month_Apr,month_May,month_Jun,month_Jul,month_Aug,month_Sep,month_Oct,month_Nov,month_Dec,month_Jan,week_day_Wednesday,week_day_Thursday,week_day_Friday,week_day_Saturday,week_day_Sunday,week_day_Monday,week_day_Tuesday
0,079XX S WABASH AVE,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,Y,623,6,08B,Feb,Wednesday,5,2020,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0000X E LOWER WACKER PL,BATTERY,DOMESTIC BATTERY SIMPLE,HOTEL/MOTEL,Y,111,42,08B,Feb,Wednesday,5,2020,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


## **Encoding hour Column**

In [53]:
column_name= "hour"

labels = list(df[column_name].unique())

labels= list(map(str,labels))
encoder_function(column_name, labels)
df.head(2)

,block,primary_description,secondary_description,location_description,domestic,beat,ward,fbi_cd,month,week_day,hour,year,primary_description_BATTERY,primary_description_THEFT,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_MOTOR VEHICLE THEFT,primary_description_WEAPONS VIOLATION,primary_description_BURGLARY,primary_description_ROBBERY,primary_description_NARCOTICS,primary_description_CRIMINAL TRESPASS,ward_28.0,ward_27.0,ward_24.0,ward_6.0,ward_8.0,ward_17.0,ward_42.0,ward_9.0,ward_21.0,ward_20.0,ward_16.0,ward_7.0,ward_34.0,ward_37.0,ward_3.0,ward_29.0,...,month_May,month_Jun,month_Jul,month_Aug,month_Sep,month_Oct,month_Nov,month_Dec,month_Jan,week_day_Wednesday,week_day_Thursday,week_day_Friday,week_day_Saturday,week_day_Sunday,week_day_Monday,week_day_Tuesday,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,hour_0,hour_1,hour_2,hour_3,hour_4
0,079XX S WABASH AVE,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,Y,623,6,08B,Feb,Wednesday,5,2020,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0000X E LOWER WACKER PL,BATTERY,DOMESTIC BATTERY SIMPLE,HOTEL/MOTEL,Y,111,42,08B,Feb,Wednesday,5,2020,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# **Deleting Label Columns**

In [54]:
data.columns

Index(['date_of_occurrence', 'block', 'iucr', 'primary_description',
       'secondary_description', 'location_description', 'arrest', 'domestic',
       'beat', 'ward', 'fbi_cd', 'latitude', 'longitude', 'month', 'week_day',
       'hour', 'year'],
      dtype='object')

In [55]:
del df["block"]
del df["primary_description"]
del df["location_description"]
del df["domestic"]
del df["beat"]
del df["fbi_cd"]
del df["month"]
del df["week_day"]
del df["hour"]
del df["ward"]

In [56]:
df.head(2)

,secondary_description,year,primary_description_BATTERY,primary_description_THEFT,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_MOTOR VEHICLE THEFT,primary_description_WEAPONS VIOLATION,primary_description_BURGLARY,primary_description_ROBBERY,primary_description_NARCOTICS,primary_description_CRIMINAL TRESPASS,ward_28.0,ward_27.0,ward_24.0,ward_6.0,ward_8.0,ward_17.0,ward_42.0,ward_9.0,ward_21.0,ward_20.0,ward_16.0,ward_7.0,ward_34.0,ward_37.0,ward_3.0,ward_29.0,ward_4.0,ward_5.0,ward_25.0,ward_10.0,block_033XX W MAYPOLE AVE,block_004XX N MICHIGAN AVE,block_006XX E 131ST ST,block_021XX E 71ST ST,block_012XX S ASHLAND AVE,block_039XX W GLADYS AVE,...,month_May,month_Jun,month_Jul,month_Aug,month_Sep,month_Oct,month_Nov,month_Dec,month_Jan,week_day_Wednesday,week_day_Thursday,week_day_Friday,week_day_Saturday,week_day_Sunday,week_day_Monday,week_day_Tuesday,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,hour_0,hour_1,hour_2,hour_3,hour_4
0,DOMESTIC BATTERY SIMPLE,2020,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,DOMESTIC BATTERY SIMPLE,2020,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# **Saving Secondary Description data**

In [57]:
df.to_csv("/content/gdrive/My Drive/Final Year Project/Dataset/preprocessed_data.csv")